In [1]:
from pandas.io.json import json_normalize #package for flattening json in pandas df
from pathlib import Path
from pytz import timezone
import pandas as pd
import json
import os 
import datetime

#function to convert json data to pandas df 
def createDF(json_data):
    try: 
        json_df = json_normalize(json_data['data'],record_path=['birds'],sep="_")
        time_stamp = datetime.datetime.strptime(json_data['timestamp'], '%d-%b-%Y %H:%M:%S.%f')
        pacific_time = time_stamp.astimezone(timezone('US/Pacific')).strftime("%d-%b-%Y %H:%M:%S.%f")
        #time_stamp = json_data['timestamp']
        bird = pd.concat([json_df.drop(['location'], axis=1), json_df['location'].apply(pd.Series)], axis=1)
        bird['timestamp'] = pacific_time

    except KeyError:
        bird = pd.DataFrame()
    return bird

In [13]:
# def createCSV():

pathlist = Path("./responses").glob('**/*.json') # return path object 
bird_data_list = []
for path in pathlist:
    path_in_str = str(path)
    with open(path_in_str) as f:
        json_f_data = json.load(f)
        json_bird_df = createDF(json_f_data)
        if json_bird_df.empty == False:
            json_bird_df = json_bird_df.rename(columns = {'id':'bird_id'})
            clean_bird_df = json_bird_df.drop(columns=['captive', 'code'])
            bird_data_list.append(clean_bird_df)
#         except ValueError: # no data for scooters, scooters are offline 
#             continue
bird_data = pd.concat(bird_data_list, axis=0)
print(bird_data.head())
print("Completed!")

   battery_level                               bird_id nest_id   latitude  \
0             63  3ffc1a8b-a939-4db4-82fa-bd44ee18d3a4     NaN  34.024433   
1             60  6137c185-4c43-4a4f-8c9c-d1be75954d4a     NaN  34.024403   
2             91  da84d4b4-d3a6-405b-8fef-21c04f31ca74     NaN  34.024963   
3             87  73f1e864-c9f9-41b4-8381-d06b12c54364     NaN  34.024591   
4             76  29b37932-e2db-41b6-a117-355b14d3791e     NaN  34.025197   

    longitude                    timestamp  
0 -118.497040  11-Jun-2019 19:35:01.455670  
1 -118.497245  11-Jun-2019 19:35:01.455670  
2 -118.496048  11-Jun-2019 19:35:01.455670  
3 -118.495383  11-Jun-2019 19:35:01.455670  
4 -118.495996  11-Jun-2019 19:35:01.455670  
Completed!


In [15]:
#bird_data.to_csv ('bird_data.csv', index = None, header=True) #output scooter data 

In [14]:
len(bird_data.bird_id.unique()) # 5089 unique birds 

5089

In [24]:
import folium
bird_data.timestamp = pd.to_datetime(bird_data.timestamp, format='%d-%b-%Y %H:%M:%S.%f')
bird_data['month'] = bird_data.timestamp.apply(lambda x: x.month)
bird_data['week'] = bird_data.timestamp.apply(lambda x: x.week)
bird_data['day'] = bird_data.timestamp.apply(lambda x: x.day)
bird_data['hour'] = bird_data.timestamp.apply(lambda x: x.hour)

In [25]:
bird_data.head()

,battery_level,bird_id,nest_id,latitude,longitude,timestamp,month,week,day,hour
0,63,3ffc1a8b-a939-4db4-82fa-bd44ee18d3a4,NaN,34.024433,-118.497040,2019-06-11 19:35:01.455670,6,24,11,19
1,60,6137c185-4c43-4a4f-8c9c-d1be75954d4a,NaN,34.024403,-118.497245,2019-06-11 19:35:01.455670,6,24,11,19
2,91,da84d4b4-d3a6-405b-8fef-21c04f31ca74,NaN,34.024963,-118.496048,2019-06-11 19:35:01.455670,6,24,11,19
3,87,73f1e864-c9f9-41b4-8381-d06b12c54364,NaN,34.024591,-118.495383,2019-06-11 19:35:01.455670,6,24,11,19
4,76,29b37932-e2db-41b6-a117-355b14d3791e,NaN,34.025197,-118.495996,2019-06-11 19:35:01.455670,6,24,11,19


In [26]:
#34.024212, -118.496475
def generateBaseMap(default_location=[34.024212, -118.496475], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [27]:
santa_monica = generateBaseMap()
santa_monica

In [33]:
from folium.plugins import HeatMap
heatmap_bird = bird_data.copy()
heatmap_bird['count'] = 1
base_map = generateBaseMap()
HeatMap(data=heatmap_bird[['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)

In [37]:
time_stamp_list = []
for hour in heatmap_bird.timestamp.sort_values().unique():
    time_stamp_list.append(heatmap_bird.loc[heatmap_bird.timestamp == hour, ['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist())


In [39]:
from folium.plugins import HeatMapWithTime
base_map = generateBaseMap(default_zoom_start=11)
HeatMapWithTime(time_stamp_list, radius=5, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_map)

In [40]:
base_map.save(os.path.join('./flaskexample/templates/', 'heatmap_santamonica_bird.html'))